In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 5]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))


[ 1  2  3  4  5  6  8 10] [954961     14 238757      4     20 238756      1      5]
(array([1, 2, 3, 5, 6]), array([954961,     14, 238757,     20, 238756]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '5' encode 3
defense '6' encode 4


In [5]:
# Collect 10% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_10, _ , y_10 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.1, stratify=y_label_0_1, random_state=42)


In [6]:
X_10.shape

(143251, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb10 = XGBClassifier()
xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [9]:


base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude3Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 6] [     1 238752]
(1, 42) (1,)
Save 2 to baseline_Def2.npy
(238752, 42) (238752,)
Save 6 to baseline_Def6.npy
Execution Time: 0.351432 seconds
(716259,)
[1 2 3 5 6] [273005     10 212517      1 230726]
(273005, 42) (273005,)
Save 1 to BIM_Def1.npy
(10, 42) (10,)
Save 2 to BIM_Def2.npy
(212517, 42) (212517,)
Save 3 to BIM_Def3.npy
(1, 42) (1,)
Save 5 to BIM_Def5.npy
(230726, 42) (230726,)
Save 6 to BIM_Def6.npy
Execution Time: 0.855159 seconds
(716259,)
[1 2 3 5 6] [479804      4   4657     33 231761]
(479804, 42) (479804,)
Save 1 to FGSM_Def1.npy
(4, 42) (4,)
Save 2 to FGSM_Def2.npy
(4657, 42) (4657,)
Save 3 to FGSM_Def3.npy
(33, 42) (33,)
Save 5 to FGSM_Def5.npy
(231761, 42) (231761,)
Save 6 to FGSM_Def6.npy
Execution Time: 0.782981 seconds
(716259,)
[1 2 3 5 6] [273005     10 212517      1 230726]
(273005, 42) (273005,)
Save 1 to PGD_Def1.npy
(10, 42) (10,)
Save 2 to PGD_Def2.npy
(212517, 42) (212517,)
Save 3 to PGD_Def3.npy
(1, 42) (1,)
Save 5 to PGD_Def5.npy
(230726, 

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude3Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[2 6] [     1 238752]
(1, 42) (1,)
Save 2 to baseline_Def2.npy
(238752, 42) (238752,)
Save 6 to baseline_Def6.npy
Execution Time: 1.241010 seconds
(716259,)
[1 3 6] [451657  25849 238753]
(451657, 42) (451657,)
Save 1 to BIM_Def1.npy
(25849, 42) (25849,)
Save 3 to BIM_Def3.npy
(238753, 42) (238753,)
Save 6 to BIM_Def6.npy
Execution Time: 2.804846 seconds
(716259,)
[1 3 6] [477336    170 238753]
(477336, 42) (477336,)
Save 1 to FGSM_Def1.npy
(170, 42) (170,)
Save 3 to FGSM_Def3.npy
(238753, 42) (238753,)
Save 6 to FGSM_Def6.npy
Execution Time: 2.542195 seconds
(716259,)
[1 3 6] [451657  25849 238753]
(451657, 42) (451657,)
Save 1 to PGD_Def1.npy
(25849, 42) (25849,)
Save 3 to PGD_Def3.npy
(238753, 42) (238753,)
Save 6 to PGD_Def6.npy
Execution Time: 2.719541 seconds
(716259,)
[1 3 6] [716209     25     25]
(716209, 42) (716209,)
Save 1 to DF_Def1.npy
(25, 42) (25,)
Save 3 to DF_Def3.npy
(25, 42) (25,)
Save 6 to DF_Def6.npy
Execution Time: 2.414329 seconds
(716259,)
[1 3 6] [44

In [11]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_Exclude3Attack/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_Exclude3Attack/WUSTL_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_Exclude3Attack.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB10,0,1,0,0,0,238752,0,0,0,0,0,238753
1,XGB10,273005,10,212517,0,1,230726,0,0,0,0,0,716259
2,XGB10,479804,4,4657,0,33,231761,0,0,0,0,0,716259
3,XGB10,273005,10,212517,0,1,230726,0,0,0,0,0,716259
4,XGB10,710560,0,5408,0,18,273,0,0,0,0,0,716259
5,XGB10,326663,46,153442,0,81,236027,0,0,0,0,0,716259
6,XGB10,1,3,0,0,0,716255,0,0,0,0,0,716259
7,XGB10,163305,32399,40002,0,2932,477621,0,0,0,0,0,716259
8,XGB10,15693,0,469887,0,1,230678,0,0,0,0,0,716259
9,XGB10,457904,0,27608,0,5,230742,0,0,0,0,0,716259
